In [8]:
import os, sys
import wandb
import tqdm.notebook as tqdm
import transformers
import torch
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import DataLoader, Dataset
from importlib import reload
from torch_scatter import scatter
from transformers import pipeline

sys.path.append('../')

from utils import preprocess as pp
from utils.ckpt import _reload_best_model
from utils.graph_llm import GraphLLM
from utils.llm import LLM
from utils.multiplex import Multiplex
from utils.textualize import *
from utils.bio_graphs import BiologicalDataset

# Load Datasets

In [9]:
# get dataset
base_path = '../data/DREAM4_gold_standards/'

c_node_id_data = BiologicalDataset(base_path + 'connections_node_id')
c_node_label_data = BiologicalDataset(base_path + 'connections_node_label')
sp_node_id_data = BiologicalDataset(base_path + 'shortest_path_node_id')
sp_node_label_data = BiologicalDataset(base_path + 'shortest_path_node_label')

# Load Models

In [6]:
vanilla_llm = LLM(max_text_len=512,
                  max_max_new_tokens=32,
                  max_memory=[80, 80],
                  llm_model_path='meta-llama/Meta-Llama-3-8B-Instruct',
                  llm_frozen='True',
                  revision="main") # need to add args

Loading LLaMA...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Freezing LLaMA...
Finished loading LLaMA...


In [10]:
# base model
base_graph_llm = GraphLLM(max_text_len=512,
                     max_max_new_tokens=32,
                     max_memory=[80, 80],
                     llm_model_path='meta-llama/Meta-Llama-3-8B-Instruct',
                     llm_frozen='True',
                     revision="main") # args are defaulted in the class

Loading LLaMA...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Freezing LLaMA!
Finished loading LLaMA!


In [13]:
# path
path = '../checkpoints/test_run/epoch_5_best.pth'

# load model
trained_graph_llm = _reload_best_model(base_graph_llm, path)

Loading checkpoint from ../checkpoints/test_run/epoch_5_best.pth.


RuntimeError: Error(s) in loading state_dict for GraphLLM:
	Missing key(s) in state_dict: "model.model.embed_tokens.weight", "model.model.layers.0.self_attn.q_proj.weight", "model.model.layers.0.self_attn.k_proj.weight", "model.model.layers.0.self_attn.v_proj.weight", "model.model.layers.0.self_attn.o_proj.weight", "model.model.layers.0.mlp.gate_proj.weight", "model.model.layers.0.mlp.up_proj.weight", "model.model.layers.0.mlp.down_proj.weight", "model.model.layers.0.input_layernorm.weight", "model.model.layers.0.post_attention_layernorm.weight", "model.model.layers.1.self_attn.q_proj.weight", "model.model.layers.1.self_attn.k_proj.weight", "model.model.layers.1.self_attn.v_proj.weight", "model.model.layers.1.self_attn.o_proj.weight", "model.model.layers.1.mlp.gate_proj.weight", "model.model.layers.1.mlp.up_proj.weight", "model.model.layers.1.mlp.down_proj.weight", "model.model.layers.1.input_layernorm.weight", "model.model.layers.1.post_attention_layernorm.weight", "model.model.layers